In [ ]:
# !pip install psycopg2
# !pip install psycopg2-binary --user
from sqlalchemy import create_engine
# engine = create_engine('postgresql://postgres@db:5432/bmt_gpt')
from sqlalchemy.engine import URL
import pandas as pd
from sqlalchemy import text
import json

from collections import Counter
import spacy

nlp = spacy.load("en_core_web_sm")

# 'NOUN', 'ADJ'
def find_types_of_words(text, word_type, nlp=nlp):
    doc = nlp(text)
    words = [x.text.lower() for x in doc if x.pos_ == word_type]
    words = list(set(words))
    words.sort()
    
    return words

def most_common_words(text_list, word_type):
    all_review_adjectives = []
    for review in text_list:
        adjs = find_types_of_words(review, word_type)
        all_review_adjectives.extend(adjs)

    most_common_words = Counter(all_review_adjectives).most_common(15)
    return most_common_words

sql_type = 'postgresql'
host = 'localhost'
# host = docker_thing("bmt-gpt-saas_default","bmt-gpt-saas_db_1")
# host = "172.18.0.1'"
port = 5432
db = 'bmt_gpt'
user = 'postgres'
pw = 'postgres'

url = URL.create(
    drivername="postgresql+psycopg2",
    username=user,
    password=pw,
    host=host,
    database=db,
    port=5432
)



In [ ]:
engine = create_engine(url)
connection = engine.connect()
# query = text('select * from "amazon_processedproductreviews" where "ASIN_ORIGINAL_ID" = ' + "'B07TVK6DC2'")
query = text('select * from "amazon_processedproductreviews" where "ASIN_ORIGINAL_ID" = ' + "'B07K32ZCHG'")
processed_product_reviews = pd.read_sql(query, connection)
all_reviews = list(processed_product_reviews['REVIEW'])

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("I bought these wonderful boxing gloves for my son")

for token in doc:
    output = {
        'token.text': token.text, 
        'token.lemma_': token.lemma_, 
        'token.pos_': token.pos_, 
        'token.tag_': token.tag_, 
        'token.dep_': token.dep_,
        'token.shape_': token.shape_,
        'token.is_alpha': token.is_alpha,
        'token.is_stop': token.is_stop
    }
#     print(json.dumps(output, indent=4))

adj = [x.text for x in doc if x.pos_=='ADJ']
print(adj)
    

In [ ]:
most_common_adjs = most_common_words(all_reviews, 'ADJ')
most_common_nouns = most_common_words(all_reviews, 'NOUN')
most_common_verbs = most_common_words(all_reviews, 'VERB')


In [ ]:
nlp = spacy.load("en_core_web_sm")
# doc = nlp("I have to go fight someone now")
doc = nlp("weekly daily monthly hourly")

for token in doc:
    output = {
        'token.text': token.text, 
        'token.lemma_': token.lemma_, 
        'token.pos_': token.pos_, 
        'token.tag_': token.tag_, 
        'token.dep_': token.dep_,
        'token.shape_': token.shape_,
        'token.is_alpha': token.is_alpha,
        'token.is_stop': token.is_stop
    }
#     print(json.dumps(output, indent=4))
#     print(json.dumps(token, indent=4))
    print(token)

In [ ]:
# !pip install keybert
from keybert import KeyBERT


doc = all_reviews[0]
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(doc)
keywords

In [ ]:
import spacy

spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

kw_model = KeyBERT(model=nlp)


In [ ]:
top_kws = [kw_model.extract_keywords(rev, keyphrase_ngram_range=(1, 8), diversity=.8, top_n=7) for rev in all_reviews[:20]]



In [ ]:
for i in range(20):
    print('~', i,' - ', top_kws[i])
# for kw in top_kws:
    # print('~', kw)

In [ ]:
for rev in all_reviews[:100]:
    print('~',rev, '\n')

In [ ]:
doc = nlp('kids')

for token in doc:
    print(token, token.lemma, token.lemma_)

In [ ]:
kw_model.extract_keywords('angry wrath sad rage', keyphrase_ngram_range=(1, 3), diversity=.5, top_n=25)

In [ ]:
import spacy
from bertopic import BERTopic
# from sklearn.datasets import fetch_20newsgroups

# spacy.cli.download("en_core_web_md")

# docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

# docs = '\n'.join(all_reviews[:100])
docs = [str(x) for x in all_reviews]

nlp = spacy.load('en_core_web_md', exclude=['tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])

topic_model = BERTopic(embedding_model=nlp)
topics, probs = topic_model.fit_transform(docs)

fig = topic_model.visualize_topics()
fig.show()

In [ ]:
# topic_model.visualize_topics()
probs

In [ ]:
import torch
from transformers import BertTokenizer, BertForQuestionAnswering

In [ ]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)


In [ ]:
def extract_information(context, question):
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    return answer

In [ ]:
review = "I'm a college student who uses this laptop every day for my assignments and video calls. It's very lightweight and portable, making it perfect for my needs. I usually use it for 6-7 hours a day. The only downside is that the battery life isn't great."

questions = [
    "Who uses the product?",
    "How often do they use the product?",
    "Where do they use the product?",
    "Why do they like the product?",
    "Why do they dislike the product?",
]

for question in questions:
    answer = extract_information(review, question)
    print(f"{question}: {answer}")

In [ ]:
[extract_information(review, question) ]

In [ ]:
for review in all_reviews[0:5]:
    print('~~~', review)
    for question in questions:
        answer = extract_information(review, question)
        print(f"{question}: {answer}")

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
def generate_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings

In [ ]:
embeddings = []
test_reviews = all_reviews[0:500]
for review in test_reviews:
    embedding = generate_embeddings(review)
    embeddings.append(embedding.reshape(-1))

In [ ]:
num_clusters = 10  # Choose the number of clusters/topics
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(embeddings)

In [ ]:
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

In [ ]:
for i in range(num_clusters):
    centroid = kmeans.cluster_centers_[i]
    similarities = cosine_similarity([centroid], embeddings)
    top_n_indices = similarities.argsort()[0][-5:]  # Top 5 reviews as examples
    print(f"Cluster {i + 1} representative reviews:")
    for index in top_n_indices:
        print('~~~', test_reviews[index])
    print("\n")

In [ ]:
most_common_adjs = most_common_words(test_reviews, 'ADJ')
most_common_nouns = most_common_words(test_reviews, 'NOUN')
most_common_verbs = most_common_words(test_reviews, 'VERB')


In [ ]:
test_thing = [x for x in test_reviews if 'comfortable' in x]
for x in test_thing:
    print(x)

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)
tfidf_matrix = vectorizer.fit_transform(test_reviews)

num_clusters = 5  # Choose the number of clusters/topics
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(tfidf_matrix)

order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

for i in range(num_clusters):
    print(f"Cluster {i + 1} top keywords:")
    for index in order_centroids[i, :10]:  # Top 10 keywords
        print(terms[index])
    print("\n")


In [ ]:
nouns = [', '.join(find_types_of_words(rev, 'NOUN', nlp=nlp)) for rev in test_reviews]

In [ ]:
nouns

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)
tfidf_matrix = vectorizer.fit_transform(nouns)

num_clusters = 10  # Choose the number of clusters/topics
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(tfidf_matrix)

order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

for i in range(num_clusters):
    print(f"Cluster {i + 1} top keywords:")
    for index in order_centroids[i, :10]:  # Top 10 keywords
        print(terms[index])
    print("\n")
